In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from diffusers import StableDiffusionLDM3DPipeline
import torch
from PIL import Image
import uuid

# Load model
pipe = StableDiffusionLDM3DPipeline.from_pretrained("Intel/ldm3d-4c")
pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Fungsi untuk generate gambar
def generate_images(prompt):
    if not prompt:
        return None, None

    output = pipe(prompt)
    rgb_image, depth_image = output.rgb[0], output.depth[0]

    # Simpan gambar dengan nama unik
    uid = str(uuid.uuid4())[:8]
    rgb_path = f"{uid}_rgb.jpg"
    depth_path = f"{uid}_depth.png"
    rgb_image.save(rgb_path)
    depth_image.save(depth_path)

    return rgb_path, depth_path

# Gradio UI
with gr.Blocks(title="LDM3D Stable Diffusion") as demo:
    gr.Markdown(
        """
        <div style="text-align: center; margin-bottom: 20px;">
            <h1 style="font-size: 2.2em;">🌀 LDM3D Stable Diffusion</h1>
            <p style="font-size: 1.1em; color: #555;">
                Masukkan prompt untuk menghasilkan gambar 3D lengkap dengan kedalaman (Depth Map) menggunakan model <b>Intel LDM3D-4C</b>.
            </p>
        </div>
        """,
        elem_id="header"
    )

    with gr.Column(elem_id="input-section", scale=1):
        prompt_input = gr.Textbox(
            label="📝 Prompt",
            placeholder="Contoh: A dragon fruit on a table",
            lines=1,
            max_lines=1,
            show_label=True
        )
        generate_btn = gr.Button("🎨 Generate Gambar", variant="primary")

    gr.Markdown("---")

    with gr.Row(elem_id="output-section"):
        rgb_output = gr.Image(
            label="📷 Gambar RGB",
            type="filepath",
            height=256,
            width=256,
            show_label=True,
            interactive=False
        )
        depth_output = gr.Image(
            label="🌊 Depth Map",
            type="filepath",
            height=256,
            width=256,
            show_label=True,
            interactive=False
        )

    generate_btn.click(fn=generate_images, inputs=prompt_input, outputs=[rgb_output, depth_output])

# Jalankan aplikasi
demo.launch()
